# Function calling

Function calling is a mechanism for "extending" an LLM and empowering it to do things it can't do on its own. For example, with function calling, you can give an LLM access to real-time flight data or real-time weather data, provide it with your current location, or enable it to use the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) to compute the distance between two locations. Function calling isn't universally supported among LLMs, but it is supported by foundation models such as `GPT-4o`, `Gemini Flash`, and `Llama`. The following example demonstrates how to implement function calling with `GPT-4o`. It uses just one function, but the same approach works with multiple functions as well.

Start by asking `GPT-4o` if it's raining in Knoxville:

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='OPENAI_API_KEY')
messages = [{ 'role': 'user', 'content': 'Is it raining in Knoxville?' }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

print(response.choices[0].message.content)

I'm unable to provide real-time weather updates. To find out if it's currently raining in Knoxville, please check a weather website, app, or local news station for the most accurate information.


Of course, it can't answer the question because it has no idea whether it's raining in Knoxville and no way to find out. Let's use function calling to fix that. The first step is to define a local function that uses an external API call to retrieve information about the weather at the specified location. This implementation uses the [OpenWeather API](https://openweathermap.org/api). To run this code, you'll need to plug in your own OpenWeather API key. API keys are free, and you get up to 1,000 calls per day for free, too:

In [ ]:
import requests, json

def get_current_weather(location):
    api_key = 'OPENWEATHER_API_KEY'
    url = f'https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=imperial'
    response = requests.get(url)
    return json.dumps(response.json())

The next step is to describe the function using a rigid JSON schema:

In [3]:
weather_tool = {
    'type': 'function',
    'function': {
        'name': 'get_current_weather',
        'description': """
            Retrieves the current weather at the specified location.
            Also returns the location's latitude and longitude and the country it's in.
            """,
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The location whose weather is to be retrieved.'
                }
            },
            'required': ['location']
        }
    }
}

For convenience, we'll define a `chat` function that calls `GPT-4o` and makes it aware of the `get_current_weather` function. `chat` returns a list of messages that includes the LLM's response, and if necessary, it uses the `get_current_weather` function to help formulate that response:

In [4]:
def chat(input, messages=None):
    if not messages:
        # Start a new message thread if none is provided
        messages = [{
            'role': 'system',
            'content': 'You are a helpful assistant who can fetch information about current weather conditions'
        }]
    
    # Add a message containing the input to this function
    message = { 'role': 'user', 'content': input }
    messages.append(message)

    # Call the LLM  and make it aware of the weather tool
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        tools=[weather_tool]
    )
    
    # If one or more tool calls are required, execute them
    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
    
            if function_name == 'get_current_weather':
                # Get the function argument(s) and call the function
                location = json.loads(tool_call.function.arguments)['location']
                output = get_current_weather(location)
    
                # Append the function output to the messages list
                messages.append({ 'role': 'function', 'name': function_name, 'content': output })
    
            else:
                raise Exception('Invalid function name')
    
        # Pass the function output to the LLM
        response = client.chat.completions.create(
            model='gpt-4o',
            messages=messages
        )
    
    # Return a message thread containing the LLM's response
    messages.append({ 'role': 'assistant', 'content': response.choices[0].message.content })
    return messages

Now use the `chat` function to find out whether it's raining in Knoxville:

In [5]:
messages = chat('Is it raining in Knoxville?')
print(messages[-1]['content'])

No, it is not currently raining in Knoxville. The weather is overcast with 100% cloud cover.


One of the features of function calling is that the LLM know what parameters each function accepts, and it know which are required parameters. In the case of the `get`current`weather` function, the LLM knows that it needs a location parameter for the function to be called. Here's what happens if you simply ask fror the weather without specifying a location:

In [6]:
messages = chat('Is it raining outside?')
print(messages[-1]['content'])

To check if it's raining, I need to know your current location. Could you please provide that information?


The LLM knows it can find out whether it's raining if you provide it with a location, so it asks for the location. You can now follow up with a message that provides that location. It's crucial in this scenario that you include previous messages in the call so the LLM "remembers" that it asked for a location:

In [7]:
messages = chat('Knoxville', messages)
print(messages[-1]['content'])

In Knoxville, it is currently overcast with clouds, but there is no indication of rain right now.


Just to be sure, let's ask the LLM a question that doesn't require a function call:

In [8]:
messages = chat('Why is the sky blue?')
print(messages[-1]['content'])

The sky appears blue primarily because of a phenomenon called Rayleigh scattering. As sunlight passes through Earth's atmosphere, it collides with molecules and small particles in the air. Sunlight consists of many colors, each with different wavelengths. 

Blue light waves are shorter and scatter more than other colors as they strike the molecules in the atmosphere. This scattered blue light is what we see when we look up at the sky during the day. The scattering causes the direct sunlight to lose some of its blue light, which is why the sun appears more yellowish when viewed directly as it gets closer to the horizon, and the sky appears blue most of the time during the day.


This is a simple example, but it demonstrates both the power and the mechanics of function calling. For more information on function caling with OpenAI models, see https://platform.openai.com/docs/guides/function-calling.